# ------------------------------------ Trading Bot --------------------------------------

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import logging #logger
#https://www.youtube.com/watch?v=SMZfgeHFFcA
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
#from network import DeepQNetwork
#from replay_memory import ReplayBuffer
#tf.get_logger().setLevel('ERROR')

import pandas as pd
import time

In [2]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [3]:
import datetime

In [4]:
from env.environment import *
from env.portfolio import *
from utils.utils import *
from agent.agent import *



In [5]:
#from pandas.tseries.holiday import USFederalHolidayCalendar

In [6]:
import psycopg2 as pg

In [7]:
from prettytable import PrettyTable

In [8]:
# »»»»»»»»»»»»»»»»» STOCKS ««««««««««««««««««««z

STOCKS={'amazon':'AMZN','apple':'AAPL','netflix':'NFLX','google':'GOOG','Accenture':'ACN',
       'alibaba':'BABA','Turtle Beach':'HEAR','Disney':'DIS',
       'LG Display':'LPL','microsoft':'MICS','sony':'SONY',
       'Cenovus Energy':'CVE','YPF':'YPF','SHELL':'SHEL','Petrobras':'PBR',
       'Coca-Cola':'KO','PespsiCo':'PEP','Unilever':'UL','Kimberly-Clark':'KMB',
       'Mondelez':'MDLZ'}

#STOCKS = {'amazon':'AMZN'}

# dir where parquete file will be created
stocks_dir = '/dataset/stocks_aux/'

#*****************************************************************

In [9]:
def turn_of_logger():
    logger=logging.getLogger()
    for handler in logger.handlers[:]:  #make a copy of the list
        logger.removeHandler(handler)
    return logger

In [10]:
strin_format='%(i)s, %(market_return)s,%(pr)s,%(Sharpe)s,%(score)s,%(avg_score)s,%(best_score)s,%(loss)s,%(n_steps)s,%(pr_val)s,%(score_val)s, %(avg_score_val)s'
def initLogging(filename, logger_name):
    logger=turn_of_logger()
    logger.setLevel(logging.DEBUG)
    formatter=logging.Formatter(fmt='%(asctime)s,%(message)s',datefmt='%Y-%m-%d,%H:%M:%S')
    #formatter=logging.Formatter(fmt='%(message)s')
    fh=logging.FileHandler(filename)
    fh.setFormatter(formatter)
    logger.addHandler(fh)
    return logger


    # sh=logging.StreamHandler(sys.stdout)
    # sh.setFormatter(formatter)
    # logger.addHandler(sh)


### ------------------------------- Reading and preparing data -----------------------------------------

In [11]:
df_training=pd.read_parquet("./get_data/data_for_model_v2.parquet")
df_training.shape

(6630, 35)

In [12]:
#check if index is sorted. if yes ==Ture
df_training.datetime.is_monotonic_increasing

True

In [13]:
df_training.head(5)

,ticker,datetime,date,hour,holidays,n_weekday,n_hour,open,high,low,...,source10_y,source9_y,source8_y,source7_y,source6_y,source5_y,source4_y,source3_y,source2_y,source1_y
3,AAPL,2022-04-06 07:00:00,2022-04-06,07:00:00,0,2,7,173.330000,173.350000,172.500000,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
4,AAPL,2022-04-06 08:00:00,2022-04-06,08:00:00,0,2,8,172.690000,175.230000,172.000000,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
5,AAPL,2022-04-06 09:00:00,2022-04-06,09:00:00,0,2,9,172.360001,173.139999,171.020004,...,0.000000,0.0,0.0,0.0,0.362349,0.0,0.0,0.000000,0.0,0.0
6,AAPL,2022-04-06 10:00:00,2022-04-06,10:00:00,0,2,10,171.130005,171.580002,170.130005,...,-0.203831,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
7,AAPL,2022-04-06 11:00:00,2022-04-06,11:00:00,0,2,11,170.839996,171.740005,170.490005,...,0.027694,0.0,0.0,0.0,0.000000,0.0,0.0,-0.000264,0.0,0.0


In [14]:
#df_training=df_training[(df_training.relevance_score>=0.5]

In [15]:
#df_training.loc[0:1,['volume']]=1

In [16]:
#Because I am not using all data, I can fill in with other value different from 0. The first value never will be 0.
df_training['volume'] = df_training['volume'].replace(0, np.nan).ffill()

In [17]:
#check 80% of split in days
split_value=int(len(df_training)*0.8)

In [18]:
df_training.iloc[split_value-1:split_value,:]['date']

14175    2023-11-17
Name: date, dtype: object

In [19]:
#filter the dates where sentiment is not available

#Select Validation first
df_validation=df_training[(df_training.date>=datetime.date(2023,11,18))&(df_training.date<=datetime.date(2024,4,25))]
print("Validation: ",df_validation.shape)
#check if index is sorted. if yes ==Ture
print(df_validation.datetime.is_monotonic_increasing)
# selece Traning second -check to improve this
df_training=df_training[(df_training.date>=datetime.date(2022,1,1))&(df_training.date<datetime.date(2023,11,17))]
print("Training: ",df_training.shape)
#check if index is sorted. if yes ==Ture
print(df_training.datetime.is_monotonic_increasing)

Validation:  (1326, 35)
True
Training:  (5291, 35)
True


In [20]:
df_validation

,ticker,datetime,date,hour,holidays,n_weekday,n_hour,open,high,low,...,source10_y,source9_y,source8_y,source7_y,source6_y,source5_y,source4_y,source3_y,source2_y,source1_y
14235,AAPL,2023-11-20 07:00:00,2023-11-20,07:00:00,0,0,7,189.840000,189.840000,189.490000,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
14236,AAPL,2023-11-20 08:00:00,2023-11-20,08:00:00,0,0,8,189.500000,190.100000,189.110000,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
14237,AAPL,2023-11-20 09:00:00,2023-11-20,09:00:00,0,0,9,189.889999,191.669006,189.880005,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
14238,AAPL,2023-11-20 10:00:00,2023-11-20,10:00:00,0,0,10,190.720001,191.369995,190.529999,...,-0.096171,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
14239,AAPL,2023-11-20 11:00:00,2023-11-20,11:00:00,0,0,11,191.169998,191.550003,191.089996,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.098929,0.0,0.0,0.162241
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17731,AAPL,2024-04-16 15:00:00,2024-04-16,15:00:00,0,1,15,169.990005,170.014999,169.279999,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
17732,AAPL,2024-04-16 16:00:00,2024-04-16,16:00:00,0,1,16,169.340000,177.000000,162.223200,...,0.000000,0.0,0.000000,0.345585,0.0,0.0,0.000000,0.0,0.0,0.000000
17733,AAPL,2024-04-16 17:00:00,2024-04-16,17:00:00,0,1,17,169.500000,169.710000,169.370000,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
17734,AAPL,2024-04-16 18:00:00,2024-04-16,18:00:00,0,1,18,169.475000,169.690000,169.380000,...,0.000000,0.0,-0.134847,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000


In [21]:
# ---------------- arrange columns-----------------------------

In [22]:
l1=list(df_training.columns)[2:55] #remove ticker, datetime 

In [23]:
l1

['date',
 'hour',
 'holidays',
 'n_weekday',
 'n_hour',
 'open',
 'high',
 'low',
 'close',
 'adj_close',
 'volume',
 'dividends',
 'stock_splits',
 'source10_x',
 'source9_x',
 'source8_x',
 'source7_x',
 'source6_x',
 'source5_x',
 'source4_x',
 'source3_x',
 'source2_x',
 'source1_x',
 'source10_y',
 'source9_y',
 'source8_y',
 'source7_y',
 'source6_y',
 'source5_y',
 'source4_y',
 'source3_y',
 'source2_y',
 'source1_y']

In [24]:
df_training=df_training[l1]

In [25]:
df_validation=df_validation[l1]

In [26]:
 #---------------------------------------------------------------------

In [27]:
def _remove_linear_tendecy (x):
    return np.ediff1d(x, to_begin=1)

In [28]:
def _simple_net_return (x):
    """
        Fractional change between the current and a prior element.
        Computes the fractional change from the immediately previous row by default. 
        This is useful in comparing the fraction of change in a time series of elements.
    """
    result=x.pct_change()
    result.fillna(0,inplace=True)
    return result

In [29]:
def _log_return (x):
    result=np.log(x/x.shift(1))
    result.fillna(0,inplace=True)
    return (result)

In [30]:

 #---------------------------------------------------------------------

In [31]:
# #df_training is a DataFrame with columns : "open", "high", "low", "close", "Volume USD"
for dataframe in ['df_training','df_validation']:
    #---------------------------------------------------------------------
    #eval(dataframe)["feature_nweek"] = eval(dataframe)["n_weekday"]
    #eval(dataframe)["feature_holiday"] =  eval(dataframe)["holidays"]
    eval(dataframe)["feature_nhour"] =  eval(dataframe)["n_hour"]
    
    #Create the feature : ( close[t] - close[t-1] )/ close[t-1]
    #df_training["diff_close"] = df_training['close'].diff()
    #eval(dataframe)["feature_close"] = ( eval(dataframe)["close"]) /  eval(dataframe)["close"].abs().max() 
    #eval(dataframe)["feature_close"] = _remove_linear_tendecy(eval(dataframe)["close"]) 
    #eval(dataframe)["feature_close"] = _simple_net_return(eval(dataframe)["close"]) 
    eval(dataframe)["feature_close"] = _log_return(eval(dataframe)["close"]) 
    
    # Create the feature : open[t] / close[t]
    eval(dataframe)["feature_open"] = _log_return(eval(dataframe)["open"]) 
    #eval(dataframe)["feature_open"] = ( eval(dataframe)["open"])/  eval(dataframe)["close"].abs().max() 
    
    # Create the feature : high[t] / close[t]
    eval(dataframe)["feature_high"] = _log_return(eval(dataframe)["high"]) 
    #eval(dataframe)["feature_high"] = ( eval(dataframe)["high"])/  eval(dataframe)["close"].abs().max() 
    
    # Create the feature : low[t] / close[t]
    eval(dataframe)["feature_low"] = _log_return(eval(dataframe)["low"]) 
    #eval(dataframe)["feature_low"]= ( eval(dataframe)["low"])/  eval(dataframe)["close"].abs().max() 
    
    #eval(dataframe)["feature_volume"] =  eval(dataframe)["volume"].apply(lambda x: np.log(x) if x!=0 else x)/  eval(dataframe)["close"].abs().max() 
    eval(dataframe)["feature_volume"] = _log_return(eval(dataframe)["volume"])     
    #eval(dataframe)["feature_volume"] =  eval(dataframe)["volume"]/  (eval(dataframe)["close"].abs().max()*1000000)
    #create columns for  relevance_score (x) and ticket_sentiment_score (y)
    # l1 is from 11 to 55 because I am selecting from score20_x to score1_y
    for column in l1[11:55]:
        column_name="feature_"+column
        eval(dataframe)[column_name]= eval(dataframe)[column] 
    #---------------------------------------------------------------------
    #eval(dataframe).dropna(inplace= True) 

/tmp/ipykernel_38452/1372185277.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eval(dataframe)["feature_nhour"] =  eval(dataframe)["n_hour"]


In [32]:
feature_list=[ x for x in list(df_training.columns) if "feature_" in x] #find all features in columns
print(feature_list)

['feature_nhour', 'feature_close', 'feature_open', 'feature_high', 'feature_low', 'feature_volume', 'feature_dividends', 'feature_stock_splits', 'feature_source10_x', 'feature_source9_x', 'feature_source8_x', 'feature_source7_x', 'feature_source6_x', 'feature_source5_x', 'feature_source4_x', 'feature_source3_x', 'feature_source2_x', 'feature_source1_x', 'feature_source10_y', 'feature_source9_y', 'feature_source8_y', 'feature_source7_y', 'feature_source6_y', 'feature_source5_y', 'feature_source4_y', 'feature_source3_y', 'feature_source2_y', 'feature_source1_y']


In [33]:

#---------------------------- check for Nulls & NAN _ relevance_score -------------------
l1=list(df_training.columns)[11:55]
serie_1=df_training[l1].isna().sum()
serie_2=df_training[l1].isnull().sum()
pd.concat( [serie_1, serie_2], join='outer',keys = ['Nan','Null'], axis=1)

,Nan,Null
dividends,0,0
stock_splits,0,0
source10_x,0,0
source9_x,0,0
source8_x,0,0
source7_x,0,0
source6_x,0,0
source5_x,0,0
source4_x,0,0
source3_x,0,0


In [34]:
#check validation notna in close columns
df_validation.feature_close.notna().sum()

1326

In [35]:
print("Training: ",df_training.shape)
print("Validation: ",df_validation.shape)

Training:  (5291, 61)
Validation:  (1326, 61)


In [36]:
df_validation

,date,hour,holidays,n_weekday,n_hour,open,high,low,close,adj_close,...,feature_source10_y,feature_source9_y,feature_source8_y,feature_source7_y,feature_source6_y,feature_source5_y,feature_source4_y,feature_source3_y,feature_source2_y,feature_source1_y
14235,2023-11-20,07:00:00,0,0,7,189.840000,189.840000,189.490000,189.500000,189.500000,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
14236,2023-11-20,08:00:00,0,0,8,189.500000,190.100000,189.110000,190.045000,190.045000,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
14237,2023-11-20,09:00:00,0,0,9,189.889999,191.669006,189.880005,190.714996,190.714996,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
14238,2023-11-20,10:00:00,0,0,10,190.720001,191.369995,190.529999,191.164993,191.164993,...,-0.096171,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
14239,2023-11-20,11:00:00,0,0,11,191.169998,191.550003,191.089996,191.229996,191.229996,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.098929,0.0,0.0,0.162241
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17731,2024-04-16,15:00:00,0,1,15,169.990005,170.014999,169.279999,169.337906,169.337906,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
17732,2024-04-16,16:00:00,0,1,16,169.340000,177.000000,162.223200,169.475000,169.475000,...,0.000000,0.0,0.000000,0.345585,0.0,0.0,0.000000,0.0,0.0,0.000000
17733,2024-04-16,17:00:00,0,1,17,169.500000,169.710000,169.370000,169.490000,169.490000,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
17734,2024-04-16,18:00:00,0,1,18,169.475000,169.690000,169.380000,169.655000,169.655000,...,0.000000,0.0,-0.134847,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000


In [37]:
#check if index is sorted. if yes ==Ture
df_training.index.is_monotonic_increasing

True

In [38]:
#Check folders to save output
import datetime
    
YEAR        = str(datetime.date.today().year)     # the current year
MONTH       = str(datetime.date.today().month)   # the current month
DATE        = str(datetime.date.today().day)      # the current day
HOUR        = str(datetime.datetime.now().hour)   # the current hour
MINUTE      = str(datetime.datetime.now().minute) # the current minute
#SECONDS     = datetime.datetime.now().second #the current second
string_folder=YEAR+MONTH+DATE+"_"+HOUR+MINUTE

In [39]:
string_folder

'2024425_2225'

In [40]:
PATH = './output/'+string_folder+'/'
if not os.path.exists(PATH):
    os.makedirs(PATH)
    # Creating folders for best DQN agents
    PATH_EVAL=PATH+'agent_best_eval_models'+'/'
    PATH_NEXT=PATH+'agent_best_next_models'+'/'
    os.makedirs(PATH_EVAL)
    os.makedirs(PATH_NEXT)
    

In [41]:

logger=initLogging(PATH+"output.log","output")
logger.info('i,market_return,pr,Sharpe,score,avg_score,best_score,loss,n_steps,pr_val,score_val, avg_score_val')

In [42]:
NUMBER_GAMES=1

In [50]:
if __name__ == '__main__':
    tf.compat.v1.disable_eager_execution() #improve 
    #manage_memory()
    # #----------------Training----------------
    # env = TradingEnv(
    #     name= "AAPL",
    #     df = df_training, # Your dataset with your custom features
    #     positions = [0, 1], # -1 (=SHORT), 0(=OUT), +1 (=LONG)
    #     portfolio_initial_value=100,
    #     initial_position=1,
        
    #     obs_columns=feature_list, #automatically set columns as input
        
    #     #trading_fees = 0.01/100, # 0.01% per stock buy / sell (Binance fees)
    #     trading_fees =0,
    #     #borrow_interest_rate= 0.0003/100, # 0.0003% per timestep (one timestep = 1h here)
    #     borrow_interest_rate= 0,
    #     verbose=1
    # )
    #----------------Validation----------------
    env_val = TradingEnv(
        name= "AAPL",
        df = df_validation, # Your dataset with your custom features
        positions = [0, 1], # -1 (=SHORT), 0(=OUT), +1 (=LONG)
        portfolio_initial_value=100,
        initial_position=1,

        obs_columns=feature_list, #automatically set columns as input
        
        trading_fees =0,
        borrow_interest_rate= 0,
        verbose=1
    )
    
    best_score = -np.inf
    load_checkpoint = False
    record_agent = False
    n_games = NUMBER_GAMES
    # Epsilon should be in 0 because I need the agent's brain tries to predidct and it must not select action by random.
    agent_brain = Agent(gamma=0.99, epsilon=1, lr=0.001,
                  #input_dims=env.observation_space.shape,n_actions=env.action_space.n, 
                  input_dims=env_val.observation_space,n_actions=len(env_val.action_space),
                  mem_size=1000000, eps_min=0.01,
                  batch_size=16, replace=1000, eps_dec=1e-3, # batch_size=32 ATTENTION
                  n_neurons1=256,n_neurons2=256,
                  dir=PATH_EVAL, dir2=PATH_NEXT,
                  env_name='Trading_bot')
    
    agent_brain.policy_net = tf.keras.models.load_model('./output/2024425_1349/model_q_eval.h5')
    print('... models loaded successfully ...')
    n_steps = 0
   
    scores, eps_history, steps_array, losses,portfolios = [], [], [], [],[]
    
    for i in range(n_games):
        done = False
        score = 0
    
        terminated = False
        truncated =False
        done=(terminated|truncated)
        #actual_obs=env.reset(seed=7)[0]# I had to do this because array is not two parameters due to new gym version
        actual_obs,info=env_val.reset(seed=395)
        print(actual_obs)
        while not done: #playing one game.
            action = agent_brain.choose_action(actual_obs,False)
            #print("env.step_action ",action)
            new_obs, reward, terminated,truncated,info = env_val.step(action)
            #----------------------------------------------------------------------
            #   New GYM version needs to create done from Terminadted and Truncated (after 200)
            done=(terminated|truncated)
            #-----------------------------------------------------------------------    
            score += reward
    
            #print(actual_obs)
            agent_brain.store_transition(actual_obs, action,reward, new_obs, done)
        
            #--------------------------------------
            #New observaction is tronsformed to actual_obs
            actual_obs = new_obs
            #-----------------------------------------
            #-----------------------------------------
            # start to train
            #loss=agent.learn()
            loss=0
            #------------------------------------------
            n_steps += 1
    
        eps_history.append(agent_brain.epsilon)
        scores.append(score)
        losses.append(loss)
        print("Score: ", score, " Loss: ", loss)
        steps_array.append(n_steps)
    
        #This is the average score. From the last 100 scores
        avg_score = np.mean(scores[-100:])
        market_return=(100*(env_val.df.loc[env_val._idx,'close'] / env_val.df.loc[0,'close'] -1))
        pr=(100*(env_val.df.loc[env_val._idx,'portfolio_valuation'] / env_val.df.loc[0,'portfolio_valuation'] -1))
        portfolios.append(pr)
        print("-------------------------------------------------------------------------------------------------------")
        print('episode {} score {:.1f} avg score {:.1f} best score {:.1f} loss {:.1f} epsilon {:.2f} steps {} MarketReturn {:.2f} PortfolioReturn {:.2f} '\
              .format(i, score, avg_score, best_score,loss, agent_brain.epsilon,n_steps,market_return,pr))
        print("-------------------------------------------------------------------------------------------------------")
        if score > best_score:
            if not load_checkpoint:
                agent_brain.save_models()
            best_score = score
        if i % 5 == 0:
            agent_brain.target_net.set_weights(agent_brain.policy_net.get_weights())
            print('Target_net has changed...')

[0, 1]
No GPU, using /device:CPU:0.
Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 256)               7680      
                                                                 
 dense_13 (Dense)            (None, 256)               65792     
                                                                 
 dense_14 (Dense)            (None, 2)                 514       
                                                                 
Total params: 73,986
Trainable params: 73,986
Non-trainable params: 0
_________________________________________________________________
No GPU, using /device:CPU:0.
Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 256)               7680      
                             

/home/marcos/.cache/pypoetry/virtualenvs/dqn-kDl4tFs4-py3.10/lib/python3.10/site-packages/keras/engine/training_v1.py:2357: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


Score:  0.287598944591025  Loss:  0
-------------------------------------------------------------------------------------------------------
episode 0 score 0.3 avg score 0.3 best score -inf loss 0.0 epsilon 1.00 steps 1325 MarketReturn -10.33 PortfolioReturn 0.29 
-------------------------------------------------------------------------------------------------------
... models saved successfully ...
Target_net has changed...


In [44]:
#aux_df.to_csv(PATH+"output_dataframe.csv")    

In [ ]:
# x = [i+1 for i in range(len(scores))]
# agent_brain.policy_net.save(PATH+'model_q_eval.h5')

In [ ]:
env.df.to_csv(f"{PATH}df_{string_folder}.csv")

In [47]:
env.df.tail(50)

NameError: name 'env' is not defined

In [48]:
env_val.df.tail(50)

,index,date,hour,holidays,n_weekday,n_hour,open,high,low,close,...,feature_source4_y,feature_source3_y,feature_source2_y,feature_source1_y,position,real_position,portfolio_valuation,Asset_value,Money_fiat,reward
1276,17605,2024-04-11,09:00:00,0,3,9,168.250000,169.169998,168.160095,168.720001,...,0.000000,0.0,0.0,0.0,0,0.0,100.287599,0.0,100.287599,0.0
1277,17606,2024-04-11,10:00:00,0,3,10,168.729996,169.419998,168.570007,169.000000,...,-0.144299,0.0,0.0,0.0,0,0.0,100.287599,0.0,100.287599,0.0
1278,17607,2024-04-11,11:00:00,0,3,11,169.001099,170.289993,169.000000,169.910004,...,0.043816,0.0,0.0,0.0,0,0.0,100.287599,0.0,100.287599,0.0
1279,17608,2024-04-11,12:00:00,0,3,12,169.919998,172.190002,169.839996,172.104996,...,0.000000,0.0,0.0,0.0,0,0.0,100.287599,0.0,100.287599,0.0
1280,17609,2024-04-11,13:00:00,0,3,13,172.110001,174.479996,171.960098,174.179993,...,0.000000,0.0,0.0,0.0,0,0.0,100.287599,0.0,100.287599,0.0
1281,17610,2024-04-11,14:00:00,0,3,14,174.179993,174.860001,173.550003,174.739899,...,0.000000,0.0,0.0,0.0,0,0.0,100.287599,0.0,100.287599,0.0
1282,17611,2024-04-11,15:00:00,0,3,15,174.740005,175.460007,174.619995,175.039993,...,0.000000,0.0,0.0,0.0,0,0.0,100.287599,0.0,100.287599,0.0
1283,17612,2024-04-11,16:00:00,0,3,16,175.040000,202.005000,155.027100,174.880000,...,0.000000,0.0,0.0,0.0,0,0.0,100.287599,0.0,100.287599,0.0
1284,17613,2024-04-11,17:00:00,0,3,17,174.880000,175.040000,174.810000,174.860400,...,0.059742,0.0,0.0,0.0,0,0.0,100.287599,0.0,100.287599,0.0
1285,17614,2024-04-11,18:00:00,0,3,18,174.860000,175.000000,174.860000,174.900000,...,0.040502,0.0,0.0,0.0,0,0.0,100.287599,0.0,100.287599,0.0
